# Function Calling
## Introduction

See <https://platform.openai.com/docs/guides/function-calling>

## Installation

In [9]:
%pip install -q openai
from openai import OpenAI
client = OpenAI()
model = "gpt-4.1"

Note: you may need to restart the kernel to use updated packages.


## Ask an LLM to check if code is secure

In [10]:
existing_code = """
console.log("Hello Patrick");
"""

completion = client.chat.completions.create(model=model, temperature=0,
    messages=[
        {
            "role": "user",
            "content": f"Check if this code is secure :\n\n{existing_code}\n\n"
        }
    ]
)

result =   completion.choices[0].message.content
print(result)


Let's review your code:

```javascript
console.log("Hello Patrick");
```

**Security Analysis:**

- This code simply prints the string "Hello Patrick" to the console.
- There are **no user inputs**, **no external data**, and **no dynamic code execution**.
- There are **no side effects** other than writing to the console.
- There is **no exposure to XSS, injection, or other common vulnerabilities**.

**Conclusion:**  
This code is **completely secure** as written. There are no security risks in this snippet.

If you have a more complex code or are concerned about a specific context (e.g., running in a web server, handling user input), please provide more details!


## Setup our own security scanner

In [11]:
def security_scanner(code: str):
    if "patrick" in code.lower():
        return "This code contains the word Patrick is therefore not safe."
    else: 
        return "This code is safe."

## Define the function schema

In [12]:
tools = [{
    "type": "function",
    "function": {
        "name": "security_scanner",
        "description": "Checks if the provided code is secure.",
        "parameters": {
            "type": "object",
            "properties": {
                "code": {
                    "type": "string",
                    "description": "The code to be analyzed for security vulnerabilities."
                },

            },
            "required": ["code"],
            "additionalProperties": False,
        },
        "strict": True # Enforce strict schema validation
    }
}]

## Launch our request

In [13]:
question = "Make sure the following code is secure and only use the information we provide to determine that: "+existing_code

messages = [{"role": "user", "content": question}]

completion = client.chat.completions.create(model=model,
    messages=messages,
    tools=tools,
)

print("Response:")

print(completion.choices[0].message.tool_calls)
print(completion.choices[0].message.content)

Response:
[ChatCompletionMessageToolCall(id='call_2lYe2B4k4fCQVnfsoXNzVgXK', function=Function(arguments='{"code":"console.log(\\"Hello Patrick\\");"}', name='security_scanner'), type='function')]
None


## Execute the function call

In [14]:
import json
tool_call = completion.choices[0].message.tool_calls[0]
args = json.loads(tool_call.function.arguments)

result = security_scanner(args["code"])
print("Security Scanner Result:")
print(result)

Security Scanner Result:
This code contains the word Patrick is therefore not safe.


## Append them to the prompt request

In [15]:
messages.append(completion.choices[0].message)  # append model's function call message
messages.append({                               # append result message
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": str(result)
})
for message in messages:
    print(message)
    print("-" * 20)

{'role': 'user', 'content': 'Make sure the following code is secure and only use the information we provide to determine that: \nconsole.log("Hello Patrick");\n'}
--------------------
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_2lYe2B4k4fCQVnfsoXNzVgXK', function=Function(arguments='{"code":"console.log(\\"Hello Patrick\\");"}', name='security_scanner'), type='function')])
--------------------
{'role': 'tool', 'tool_call_id': 'call_2lYe2B4k4fCQVnfsoXNzVgXK', 'content': 'This code contains the word Patrick is therefore not safe.'}
--------------------


## Ask for a new answer

In [16]:
completion_2 = client.chat.completions.create(
    model="gpt-4.1",
    messages=messages,
    tools=tools,
)
print("Response:")
print(completion_2.choices[0].message.content)

Response:
Based on the information and analysis provided, the code console.log("Hello Patrick"); is considered not safe because it contains the word "Patrick". No further security concerns are evident from the code itself, but the presence of "Patrick" in the output is flagged as a security issue according to the given criteria.
